In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random


df = pd.read_csv("/Users/veillonpaul-armand/Documents/Projets/Data/Carto/COMARR_conso.csv",sep = ';')
print(df.head(5)) 

df2 = pd.read_csv('/Users/veillonpaul-armand/Documents/Projets/Data/Carto/correspondance-code-insee-code-postal.csv',sep=";")
print(df2.head(5)) 
df2=df2.drop(["Code Canton","Code Arrondissement","ID Geofla","Commune","Département","Région"],axis=1)



  CODGEO                   LIBGEO DEP NATURE_EPCI  ZE2010 UU2010  TUU2015  \
0   1001  L'Abergement-Clémenciat   1          CC    8213   1000        0   
1   1002    L'Abergement-de-Varey   1          CC    8201   1000        0   
2   1004        Ambérieu-en-Bugey   1          CC    8201   1302        3   
3   1005      Ambérieux-en-Dombes   1          CC    8213   1000        0   
4   1006                  Ambléon   1          CC    8216   1000        0   

   TDUU2015 AU2010  TAU2015  CATAEU2010 BV2012  
0         5    997        0         120   1093  
1         4      2        9         112   1004  
2        32      2        9         112   1004  
3         6      2        9         112  69123  
4         3    998        0         300   1034  
  Code INSEE Code Postal                Commune     Département  \
0      32460       32720             VERGOIGNAN            GERS   
1      51141       51240  LA CHAUSSEE-SUR-MARNE           MARNE   
2      77130       77580              COUL

In [2]:
#Merge on df
df=df.merge(df2,how="right",left_on="CODGEO", right_on="Code INSEE")

#Pour les codes postales avec plusieur communes, assignés au CP la population moyenne pondérée des villages
df['Counts'] = df.groupby(['Code Postal'])['CODGEO'].transform('count')
df['POP_CP'] = df.groupby(['Code Postal'])['Population'].transform('sum')
df['POP_mean_CP']=df['Population'] **2/df['POP_CP']
df['POP_mean_CP']=df.groupby(['Code Postal'])['POP_mean_CP'].transform('sum')
df.head(5)




,CODGEO,LIBGEO,DEP,NATURE_EPCI,ZE2010,UU2010,TUU2015,TDUU2015,AU2010,TAU2015,...,Statut,Altitude Moyenne,Superficie,Population,Code Commune,Code Département,Code Région,Counts,POP_CP,POP_mean_CP
0,1001,L'Abergement-Clémenciat,1,CC,8213.0,1000,0.0,5.0,997,0.0,...,Commune simple,242.0,1565.0,0.8,1,1,82,10,11.0,2.654545
1,1002,L'Abergement-de-Varey,1,CC,8201.0,1000,0.0,4.0,2,9.0,...,Commune simple,483.0,912.0,0.2,2,1,82,4,4.1,1.631707
2,1004,Ambérieu-en-Bugey,1,CC,8201.0,1302,3.0,32.0,2,9.0,...,Chef-lieu canton,379.0,2448.0,13.4,4,1,82,8,22.6,8.618584
3,1005,Ambérieux-en-Dombes,1,CC,8213.0,1000,0.0,6.0,2,9.0,...,Commune simple,290.0,1605.0,1.6,5,1,82,8,7.8,2.782051
4,1006,Ambléon,1,CC,8216.0,1000,0.0,3.0,998,0.0,...,Commune simple,589.0,602.0,0.1,6,1,82,23,19.1,4.298953


In [3]:
df['CP_TUU']=df.groupby(['Code Postal'])['TUU2015'].transform("max")
print(df.groupby(['CP_TUU'])['CODGEO'].count())

df=df[["Code Région","Code Département","Code Postal","Code INSEE","Counts","POP_CP","POP_mean_CP","CP_TUU","LIBGEO"]]
df=df.drop_duplicates(["Code Région","Code Département","Code Postal"])



CP_TUU
0.0    14535
1.0     7315
2.0     4607
3.0     2500
4.0     2210
5.0     1181
6.0      766
7.0     1759
8.0      523
Name: CODGEO, dtype: int64


In [4]:

df.groupby(['CP_TUU'])['Code INSEE'].count()
vec=df.loc[df["Code Postal"].str.find("/")!=(-1),"Code Postal"]




KeyError: 'Column not found: CODGEO'

In [ ]:
vec=df.loc[df["Code Postal"].str.find("/")!=(-1),"Code Postal"]
n=df.shape[0]

for i in vec:
    cp=i.split("/")
    df.iloc[n]=df.loc[df["Code Postal"]==i,:]
    n=n+1
    

In [ ]:
df["Code Postal"].str.find("/")